In [ ]:
import torch
import torch.nn as nn
import torch.optim as potim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

In [ ]:
# input datas
train_dataset.data.shape, test_dataset.data.shape

# output datas
train_dataset.targets.shape, test_dataset.targets.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_size = 4
for it_sam, _sample in enumerate(train_dataset.data[:10]):

    cur_idx = it_sam%plot_size+1

    plt.subplot(1, plot_size, cur_idx)

    _sample_target = train_dataset.targets[it_sam]
    plt.title(_sample_target.item())
    plt.imshow(_sample, 'gray')


    if cur_idx == plot_size:
        plt.show()

In [ ]:
# input normalize, mini-batch
sample_ = next(iter(train_loader))

In [ ]:
train_dataset.data[0]

In [ ]:
sample_[0].shape # input # sample_[1] # target

In [ ]:
sample_[1].dtype

In [ ]:
class mnist_fcn(nn.Module):
  def __init__(self, input_size, output_size=10, layers=[120, 84]):
    super(mnist_fcn, self).__init__()
    self.fc1 = nn.Linear(input_size, layers[0])
    self.fc2 = nn.Linear(layers[0], layers[1])
    self.fc3 = nn.Linear(layers[1], output_size)

    self.relu = nn.ReLU(inplace=True)

  def forward(self, x):
    o = self.relu(self.fc1(x))
    o = self.relu(self.fc2(o))
    o = self.fc3(o)

    return o

In [ ]:
model = mnist_fcn(784, 1) # categori 3 [0, 0, 0, 1, 0, 0, 0, ] --> MSE 0~9
print(model)

In [ ]:
# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = 'mps'


In [ ]:
# criterion = nn.CrossEntropyLoss() # Categori class, 0 1 2 3
criterion = nn.MSELoss() # Linear
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
epochs= 5
model = model.to(device)
for e in range(epochs):

  for it_batch, (images, labels) in enumerate(train_loader):
    # 100, 1, 28, 28 --> 100, 784(28*28*1)
    images = images.reshape(-1, 784)
    images = images.to(device) # if gpu is ture : cpu --> gpu

    labels = labels.reshape(-1, 1)
    labels = labels.type(torch.float32)
    labels = labels.to(device)

    optimizer.zero_grad()
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    if (it_batch+1) % 100 == 0:
      print(f'epoch:{e}, batch:{it_batch+1}, curr_loss:{loss.item()}')
      with torch.no_grad():
        model.eval()
        total_sum = 0
        correct_sum =0
        for it_batch, (images, labels) in enumerate(test_loader):
          images = images.reshape(-1, 784)
          images = images.to(device) # if gpu is ture : cpu --> gpu
          labels = labels.reshape(-1, 1)
          labels = labels.type(torch.float32)
          labels = labels.to(device)
          total_sum += images.shape[0]


          outputs = model(images)
          # pred = torch.argmax(outputs, axis=1)
          # correct_sum += (pred == labels).sum().item()
          correct_sum += torch.sum(torch.round(outputs) == labels).item()

        print(f'correct:{correct_sum}, total:{total_sum}, accuracy {correct_sum/total_sum*100} ')


      model.train()


In [ ]:
model = model.to('cpu')

In [ ]:
test_sample = next(iter(test_loader))

In [ ]:
outputs = model(test_sample[0].reshape(-1, 784))

In [ ]:
torch.sum(torch.round(outputs[0:10]) == test_sample[1][:10].reshape(-1, 1)).item()

In [ ]:

with torch.no_grad():
  model.eval()
  total_sum = 0
  correct_sum =0
  for it_batch, (images, labels) in enumerate(test_loader):
    images = images.reshape(-1, 784)
    total_sum += images.shape[0]

    outputs = model(images)
    pred = torch.argmax(outputs, axis=1)
    correct_sum += (pred == labels).sum().item()

  print(f'correct:{correct_sum}, total:{total_sum}, accuracy {correct_sum/total_sum*100} ')



In [ ]:
# 모델 결과 시각화 ( 이미지 target )

# 3행 3열 9개 데이터 시각화, title에 정답, 예측값 표시. 이미지 형식(28, 28)로

# 행 : 5, 열 : 20 테스트 데이터 시각화, ''

In [ ]:
tmp = iter(test_loader)
images, labels = next(tmp)
print(images.shape)

In [ ]:
import numpy as np
with torch.no_grad():
  model.to('cpu')
  model.eval()
  y_pred = model(images.reshape(-1, 28*28))
  y_pred2 = np.argmax(y_pred.numpy(), axis=1)

In [ ]:
def display_test(x, y, y_pred):
  plt_r = 5
  plt_c = 20
  plt.figure(figsize=(15, 5))
  for y_pos in range(plt_r):
    for x_pos in range(plt_c):
      idx = y_pos*plt_c + x_pos
      plt.subplot(plt_r, plt_c, idx+1)
      plt.title('[%d, %d]' % (y[idx], y_pred[idx]))
      plt.imshow(x[idx].reshape(28, 28), 'gray')
      ax = plt.gca()
      ax.axes.xaxis.set_visible(False)
      ax.axes.yaxis.set_visible(False)
  plt.show()

In [ ]:
display_test(images, labels, y_pred2)